In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
import nltk
import time, os, pickle
from nltk.tokenize import word_tokenize, sent_tokenize
from itertools import combinations, permutations
# plt.style.use('ggplot')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# import data

In [2]:
df1 = pd.read_csv(r'../all-the-news/articles1.csv')
df2 = pd.read_csv(r'../all-the-news/articles2.csv')
df3 = pd.read_csv(r'../all-the-news/articles3.csv')
df = df1.append(df2).append(df3)
df.index = range(df.shape[0])
del df1, df2, df3
df.shape

(142570, 10)

In [3]:
df.sample(5)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
132003,135466,204709,White House says Mexico border wall might be f...,Washington Post,Joshua Partlow,2017-01-26,2017.0,1.0,https://web.archive.org/web/20170127000123/htt...,MEXICO CITY — A deep rift opened Thursday...
106518,109978,164052,"Student Loans: You’ve Got Questions, We’ve Got...",NPR,Anya Kamenetz,2017-04-23,2017.0,4.0,http://www.npr.org/sections/ed/2017/04/23/5250...,With student debt at a staggering $1. 3 trilli...
76904,80361,121330,Trump Is Losing because He’s Losing — Not beca...,National Review,Kevin D. Williamson,2016-10-19,2016.0,10.0,http://www.nationalreview.com/article/441193/d...,Donald Trump protests that his ascent to the W...
7801,7801,26537,"U.S. Presses for Truce in Syria, With Its Larg...",New York Times,Mark Landler and Mark Mazzetti,2017-03-08,2017.0,3.0,NaN,"HANGZHOU, China — The image of a Syrian ..."
43800,46616,65717,Trump announces he won’t attend this year’s Wh...,Business Insider,NaN,2017-02-26,2017.0,2.0,NaN,’ ’ ” President Donald Trump on Saturday t...


In [4]:
df.publication.unique()

array(['New York Times', 'Breitbart', 'CNN', 'Business Insider',
       'Atlantic', 'Fox News', 'Talking Points Memo', 'Buzzfeed News',
       'National Review', 'New York Post', 'Guardian', 'NPR', 'Reuters',
       'Vox', 'Washington Post'], dtype=object)

# Define bias of each publication based on mediabiasfactcheck.com

In [5]:
# actual labels from MBFC
bias_dict = {
    'New York Times': 'left-center',
    'Breitbart': 'extreme-right',
    'CNN': 'left',
    'Business Insider': 'left-center',
    'Atlantic': 'left-center',
    'Fox News': 'right',
    'Talking Points Memo': 'left',
    'Buzzfeed News': 'left-center',
    'National Review': 'right',
    'New York Post': 'right-center',
    'Guardian': 'left-center',
    'NPR': 'left-center',
    'Reuters': 'neutral',
    'Vox': 'left',
    'Washington Post': 'left-center'
}

In [6]:
# simplified labels
for k,v in bias_dict.items():
    if 'left' in v:
        bias_dict[k] = 'liberal'
    elif 'right' in v:
        bias_dict[k] = 'conservative'
    else:
        bias_dict[k] = 'neutral'
for k,v in bias_dict.items():
    print(k,v)

New York Times liberal
Breitbart conservative
CNN liberal
Business Insider liberal
Atlantic liberal
Fox News conservative
Talking Points Memo liberal
Buzzfeed News liberal
National Review conservative
New York Post conservative
Guardian liberal
NPR liberal
Reuters neutral
Vox liberal
Washington Post liberal


In [7]:
# load in the bigrams and lexicons
datapath = r'../data'
# with open(os.path.join(datapath, 'libCons.pickle'), 'rb') as f:
#     libCons = pickle.load(f)
# with open(os.path.join(datapath, 'conCons.pickle'), 'rb') as f:
#     conCons = pickle.load(f)
# with open(os.path.join(datapath, 'libGrams.pickle'), 'rb') as f:
#     libGrams = pickle.load(f)
# with open(os.path.join(datapath, 'conGrams.pickle'), 'rb') as f:
#     conGrams = pickle.load(f)
with open(os.path.join(datapath, 'libCons2.pickle'), 'rb') as f:
    libCons = pickle.load(f)
with open(os.path.join(datapath, 'conCons2.pickle'), 'rb') as f:
    conCons = pickle.load(f)
with open(os.path.join(datapath, 'libGrams2.pickle'), 'rb') as f:
    libGrams = pickle.load(f)
with open(os.path.join(datapath, 'conGrams2.pickle'), 'rb') as f:
    conGrams = pickle.load(f)

In [8]:
len(libCons), len(libGrams), len(conCons), len(conGrams)

(23, 85, 9, 63)

# testing

In [93]:
s = ['the']
len(set(s).intersection(libCons)) + len(set(s).intersection(conCons))

0

In [100]:
def extractContent(row,n=None):
    sentList = []
#     label = bias_dict[row.publication]
    try:
        sentences = sent_tokenize(row.content)
    except:
        sentences = []
    sentences.append(row.title)
    for s in sentences:
        try:
            words = word_tokenize(s)
            words = [w.lower() for w in words]
            bigrams = []
            for i in range(len(words) - 1):
                bigrams.append(' '.join((words[i], words[i+1])))
        except:
            words = []
        libConsPresent = set(words).intersection(libCons)
        libGramsPresent = set(bigrams).intersection(libGrams)
        conConsPresent = set(words).intersection(conCons)
        conGramsPresent = set(bigrams).intersection(conGrams)
        libNum = len(libConsPresent) + len(libGramsPresent)
        conNum = len(conConsPresent) + len(conGramsPresent)
        if libNum == conNum:
            label = 'neutral'
        elif libNum > conNum:
            label = 'liberal'
        else:
            label = 'conservative'
        sentList.append((label, s))
#         if label == 'liberal':
#             if n:
#                 lexPresent = set(words).intersection(libCons[:n])
#                 gramsPresent = set(bigrams).intersection(libGrams[:n])
#             else:
#                 lexPresent = set(words).intersection(libCons)
#                 gramsPresent = set(bigrams).intersection(libGrams)
#             if lexPresent and gramsPresent:
#                 sentList.append((label, s, lexPresent, gramsPresent))
#         elif label == 'conservative':
#             if n:
#                 lexPresent = set(words).intersection(conCons[:n])
#                 gramsPresent = set(bigrams).intersection(conGrams[:n])
#             else:
#                 lexPresent = set(words).intersection(conCons)
#                 gramsPresent = set(bigrams).intersection(conGrams)
#             if lexPresent and gramsPresent:
#                 sentList.append((label, s, lexPresent, gramsPresent))
#         else:
#             lexPresent = set(words).intersection(set(libCons).union(conCons))
#             gramsPresent = set(bigrams).intersection(set(libGrams).union(conGrams))
#             if not lexPresent and not gramsPresent:
#                 sentList.append((label, s, lexPresent, gramsPresent))
    return sentList

In [101]:
sample = df.loc[df.publication == 'Breitbart'].sample().iloc[0]
sample

Unnamed: 0                                                 10169
id                                                         28905
title          MILO: Trump Slims Down Lena Dunham. Is There A...
publication                                            Breitbart
author                                                      Milo
date                                                  2017-02-07
year                                                        2017
month                                                          2
url                                                          NaN
content        Lena Dunham claims (claims!) she is losing wei...
Name: 10169, dtype: object

In [102]:
sentList = extractContent(sample)
len(sentList), len(sent_tokenize(sample.content))

(53, 52)

In [105]:
for i,s in enumerate(sentList):
    if s[0] != 'neutral':
        print(i,s)

28 ('conservative', 'In the past she was content to be a typical feminist    and an ugly poster child for the body positivity movement.')
29 ('liberal', 'But since the election she has vomited forth a steady stream of hysteria and worse.')


In [104]:
for i,s in enumerate(sent_tokenize(sample.content)):
    print(i,s)

0 Lena Dunham claims (claims!)
1 she is losing weight because Donald Trump has convinced her not to eat.
2 The Catholic church won’t declare a person a saint until after their death, but if this extraordinary statement is true, maybe a miracle of this magnitude deserves an exception?
3 [Dunham revealed the news on the Howard Stern Show, during which the actress went through her tired   routine of explaining that promising to move to Canada was “just a joke.
4 ” She also said Hillary Clinton was “the best qualified candidate we’ve ever had.
5 ”  But in the middle of her schtick, an amazing revelation emerged.
6 Lena said “Donald Trump became president and I stopped being able to eat food  …    Everyone’s been asking like, ‘What have you been doing?’ And I’m like, try   pain and devastation and hopelessness and you, too, will lose weight.
7 ” Now that we’ve collectively pulled ourselves together, stopped guffawing and picked the sick out of our keyboards, let’s unpack that statement for 

In [109]:
# testing
sample = df.sample(20)
sents = []
for i in range(20):
    sentList = extractContent(sample.iloc[i])
    sents.extend(sentList)
df2 = pd.DataFrame(sents, columns=['label','text'])
df2[df2.label != 'neutral'].shape

(32, 2)

In [111]:
for i,t in enumerate(df2[df2.label != 'neutral'].values):
    print(i,t)

0 ['conservative'
 'It’s believed that the   tomb at the heart of the church was where the body of Jesus Christ was once laid.']
1 ['conservative'
 'My knees are shaking a little bit because I wasn’t expecting this,” Fredrik Hiebert, National Geographic’s    is quoted by the publication’s website.']
2 ['conservative'
 'Today, Egypt’s current president Abdel Fattah   is many of the things Mubarak never was: ambitious, zealous, unforgiving, coupled with that sense of grandeur and destiny that the most brutal autocrats always seem to possess.']
3 ['conservative' '.. and this area is enclosed with a chained fence.']
4 ['conservative'
 'Holloway: Well I kept looking at it and looking at the facts and what he said, and I thought, man, I can’t go here because I’ve gone through so many of these where they had details and facts and it turned out to be nothing.']
5 ['liberal'
 'We were kind of blocked in the areas that we wanted to do the search so it was kind of a failed mission I guess, so to 

In [114]:
def extractSentences(df,n=None):
    output = []
    for i in range(df.shape[0]):
        if n:
            sentList = extractContent(df.iloc[i],n)
        else:
            sentList = extractContent(df.iloc[i])
        output.extend(sentList)
    df = pd.DataFrame(output, columns=['label','text'])
    return df

In [ ]:
# start = time.time()
# df1 = extractSentences(df,50)
# print('runtime (min): ', (time.time() - start) / 60)
# df1.groupby('label').count()

In [115]:
start = time.time()
df2 = extractSentences(df)
print('runtime (min): ', (time.time() - start) / 60)
df2.groupby('label').count()

runtime (min):  19.849251770973204


,text
label,
conservative,124818
liberal,292533
neutral,4795360


In [116]:
sample = df2[df2.label != 'neutral'].sample(20)
for i,t in enumerate(sample.values):
    print(i,t)

0 ['liberal'
 '.. gives them that stamp of approval showing ’’if a big name brand designer is willing to collaborate with us and give us that stamp of approval,’’ that kind of trickles down to the customer and their perception of ’’it’’s okay to still shop here even if I [have] issues with [the company].']
1 ['liberal'
 'News institutions also serve a very valuable role for the Left in providing a place for people who have similar values to remain connected to the larger movement while still maintaining the independence from parties and advocacy groups that they desire.']
2 ['conservative'
 'I think everybody needs to understand: All people need is opportunity, and everyone’s willing to change with a little bit of hard work,” LaCourt says.']
3 ['liberal'
 'I love those people  …     It’s a big deal,” Trump said, noting that the reason they are turning to him is because “Hillary Clinton is the wrong messenger.']
4 ['liberal'
 'Unless you turn to the topic of student loans, in which case

In [ ]:
# with open(r'../data/filteredNews_top50grams.pickle', 'wb') as f:
#     pickle.dump(df1, f, pickle.HIGHEST_PROTOCOL)
with open(r'../data/filteredNews_allgrams.pickle', 'wb') as f:
    pickle.dump(df2, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(r'../data/filteredNews_top50grams.pickle', 'rb') as f:
#     df1 = pickle.load(f)
with open(r'../data/filteredNews_allgrams.pickle', 'rb') as f:
    df2 = pickle.load(f)

In [ ]:
df2.shape

In [12]:
!pwd

/home/alex/Documents/MIDS/w266/final_project/w266_final_project


In [17]:
df.to_csv('filtered_sentences.csv')

In [18]:
import pickle

In [26]:
start = time.time()
with open('filtered_sentences.pickle', 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)
with open('filtered_sentences.pickle', 'rb') as f:
    df2 = pickle.load(f)
time.time() - start

0.7255961894989014

In [27]:
start = time.time()
df.to_csv('filtered_sentences.csv')
df3 = pd.read_csv('filtered_sentences.csv')
time.time() - start

4.2353599071502686

In [24]:
with open('filtered_sentences.pickle', 'rb') as f:
    df2 = pickle.load(f)

In [25]:
df2.head()

,label,text
0,liberal,WASHINGTON — Congressional Republicans have...
1,liberal,The incoming Trump administration could choose...
2,liberal,"In another twist, Donald J. Trump’s administra..."
3,liberal,“Given that this pending litigation involves t...
4,liberal,"“Upon taking office, the Trump administration ..."
